In [49]:
import cv2
import numpy as np
import os

import numpy as np

def calculate_orientation(pts):
    size = len(pts)
    data_pts = np.empty((size, 2), dtype=np.float64)
    for i in range(data_pts.shape[0]):
        data_pts[i, 0] = pts[i, 0, 0]
        data_pts[i, 1] = pts[i, 0, 1]
    centroid = np.mean(data_pts, axis=0)
    data_pts -= centroid
    cov_mat = np.cov(data_pts.T)
    eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)
    angle = np.arctan2(eigen_vecs[0, 1], eigen_vecs[0, 0])
    return angle

def align_shapes(template_image_path, shape_image_folder):
    # Load the template image
    template_image = cv2.imread(template_image_path)
    template_gray = cv2.cvtColor(template_image, cv2.COLOR_BGR2GRAY)
    _, template_bw = cv2.threshold(template_gray, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(template_bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    angle_template = 0
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if area < 1000:
            continue
        angle_template = calculate_orientation(contour)
        break

    # Get the paths to all shape images in the folder
    shape_paths = [os.path.join(shape_image_folder, img_name) for img_name in os.listdir(shape_image_folder)]
    shape_names = [img_name.split("/")[-1] for img_name in shape_paths]

    # Load and align the shapes
    aligned_shapes = []
    for shape_image_path, shape_name in zip(shape_paths, shape_names):
        # Load the shape image
        shape_image = cv2.imread(shape_image_path)
        shape_gray = cv2.cvtColor(shape_image, cv2.COLOR_BGR2GRAY)
        _, shape_bw = cv2.threshold(shape_gray, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(shape_bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        angle_shape = 0
        for i, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if area < 1000:
                continue
            angle_shape = calculate_orientation(contour)
            break

        # Calculate difference angle between the template shape and the current shape
        angle_diff = np.degrees(angle_shape - angle_template)

        # Rotate the current shape by the difference angle
        height, width = shape_image.shape[:2]
        center = (width // 2, height // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle_diff, 1.0)
        aligned_shape = cv2.warpAffine(shape_image, rotation_matrix, (width, height))
        aligned_shapes.append(aligned_shape)

        # Save the aligned shape image
        aligned_shape_path = os.path.join("aligned", "aligned_" + shape_name)
        cv2.imwrite(aligned_shape_path, aligned_shape)

    return aligned_shapes, shape_names

aligned_shapes, shape_names = align_shapes("Templete_Input.png", "KIMIA dataset")


In [50]:
import os

def save_aligned_images(aligned_shapes, output_folder, shape_names):
    # Create the output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Save each aligned shape as a PNG image
    for i, aligned_shape in enumerate(aligned_shapes):
        output_path = os.path.join(output_folder, f"aligned_shape_{shape_names[i]}.png")
        cv2.imwrite(output_path, aligned_shape)

# Call the save_aligned_images function after the align_shapes function
output_folder = "aligned"
save_aligned_images(aligned_shapes, output_folder, shape_names)


- The question then creates a new folder named `aligned`.

+---------------------+            +-----------------------+
|  Start: Load Images |            |                       |
|    template & shapes|            |                       |
+----------+----------+            |                       |
           |                      |                       |
           v                      |                       |
+---------------------+            |                       |
|   Process Template  |            |                       |
|   (Find orientation)|            |                       |
+----------+----------+            |                       |
           |                      |                       |
           v                      |                       |
+---------------------+            |                       |
|   Loop through      | No         |                       |
|   Shape Images      +------------>                       |
|   in the folder     |            |                       |
+----------+----------+            |                       |
           | Yes                  |                       |
           v                      |                       |
+---------------------+            |                       |
|   Process Shape     |            |                       |
|   (Find orientation)|            |                       |
+----------+----------+            |                       |
           |                      |                       |
           v                      |                       |
+---------------------+            |                       |
|   Calculate Angle   |            |                       |
|   Difference        +------------>                       |
+----------+----------+            |                       |
           |                      |                       |
           v                      |                       |
+---------------------+            |                       |
|   Rotate Shape       |            |                       |
|   (Warp Affine)     +------------>                       |
+---------------------+            |                       |
           |                      |                       |
           v                      |                       |
+---------------------+            |                       |
|   Save Aligned Shape |            |                       |
|   to Output Folder  +------------>                       |
+----------+----------+            |                       |
           |                      |                       |
           v                      |                       |
+---------------------+            |                       |
|  Loop End?          +------------>                       |
+----------+----------+            |                       |
           | No                   |                       |
           v                      |                       |
+---------------------+            |                       |
|  Save All Aligned    |            |                       |
|  Shapes as PNG      +------------>                       |
+---------------------+            |                       |
           |                      |                       |
           v                      |                       |
+---------------------+            |                       |
|  End: Program Finish|            |                       |
+---------------------+            |                       |
                                   |                       |
                                   +-----------------------+